In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
# url of the table
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# read the url and get the html content
t = requests.get(url)
html_content = t.content
# beautifulsoup magic - the table is identified by it class
html_soup = BeautifulSoup(html_content, 'html.parser')
sov_tables = html_soup.find('table', class_="wikitable sortable")
#on to pandas dataframe
df = pd.read_html(str(sov_tables))
df = df[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [44]:
#remove lines where Borough in not assigned
df1 = df[df.Borough != 'Not assigned']
df1.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [45]:
#use comma as separator for multi neighborhood boroughs
df2 =df1 
df2.replace(to_replace=" / ", value=", ", regex=True,inplace=True)
df2.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [46]:
#check on Neighborhoods = NaN
df2[(df2['Neighborhood'] == "NaN")]

,Postal code,Borough,Neighborhood


In [47]:
#end of question 1
df2.shape

(103, 3)

In [59]:
## Question 2
url_code='https://cocl.us/Geospatial_data'
df_latlng = pd.read_csv(url_code)
df_latlng.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:
# a bit of tweaking to perform the merge
df_latlng.rename(columns = {'Postal Code':'Postal code'}, inplace = True)
df_latlng.head()


,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [61]:
df_full = pd.merge(df2, df_latlng, on='Postal code')

df_full.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [64]:
#clustering on the map
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium # plotting library



     |████████████████████████████████| 92kB 14.2MB/s eta 0:00:01


AttributeError: module 'folium.features' has no attribute 'CircleMarker'

In [73]:
latitude=43.7
longitude=-79.32
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Toronto

toronto_map

In [74]:


# add boroughs to the map as blue  markers
for lat, lng, label in zip(df_full.Latitude, df_full.Longitude, df_full.Borough):
    folium.features.Marker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(toronto_map)

# display map
toronto_map

In [75]:
from sklearn.cluster import KMeans

In [ ]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)